In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
import scipy
from scipy.stats import pearsonr
import sklearn
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
raw_df = pd.read_csv('drive/MyDrive/MLGroupProject/MarketingData/bank-full.csv', sep=';')
raw_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
base_df = raw_df.rename(columns={
    'default': 'has_defaulted',
    'pdays': 'days_since_last_contact',
    'previous': 'prev_contact_count',
    'poutcome': 'prev_outcome',
    'y': 'outcome'})

base_df.head()

,age,job,marital,education,has_defaulted,balance,housing,loan,contact,day,month,duration,campaign,days_since_last_contact,prev_contact_count,prev_outcome,outcome
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
# Boolean columns
bool_cols = ['has_defaulted', 'housing', 'loan', 'outcome']

# Categorical columns including boolean
cat_cols = ['job', 'marital', 'education', 'contact', 'month', 'prev_outcome'] + bool_cols

# Numeric columns
num_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'days_since_last_contact', 'prev_contact_count']

# List of unique values in each categorical column.
cat_labels = {
    'job': ['unknown', 'unemployed', 'student', 'retired', 'self-employed', 'housemaid', 'technician', 'entrepreneur', 'blue-collar', 'admin.', 'services', 'management'],
    'marital': ['unknown', 'single', 'divorced', 'married'],
    'education': ['unknown', 'primary', 'secondary', 'tertiary'],
    'contact': ['unknown', 'cellular', 'telephone'],
    #TODO: Should month be numeric?
    #TODO: Should month/date even be included? it's unlikely to be relevant to
    # bank balance. We could actually strip out a lot of these featues without
    # losing much.
    'month': ['unknown', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
    'prev_outcome': ['unknown', 'failure', 'success', 'other']
}
for col in bool_cols:
    cat_labels[col] = ['no', 'yes']

# Map equivalent of cat_labels.
cat_map = {}
for col in cat_labels:
    cat_map[col] = {cat: i for i, cat in enumerate(cat_labels[col])}

In [10]:
# Replace -1's with N/A
for col in num_cols:
    # Balance may be negative.
    if col in ['balance']: continue
    # -1's mean that the data isn't available. In those cases it's common to
    # substitute the mean value for the column rather than having a hard-coded
    # value. This is needed for simpler models like linear regression to have
    # good accuracy as we want the presence of absence of each point to have as
    # little effect on the learnt model as possible.

    # We want to replace all -1's the the mean value. This is done in two steps:
    #   1. Replace -1's with N/A's.
    #   2. Replace the N/A's with the calculated means.
    # This can't be done in one step because we must remove the -1's from the
    # column to calculate the mean.
    base_df[col] = base_df[col].replace(-1, pd.NA)
    base_df[col] = base_df[col].fillna(base_df[col].mean())

base_df.head()

,age,job,marital,education,has_defaulted,balance,housing,loan,contact,day,month,duration,campaign,days_since_last_contact,prev_contact_count,prev_outcome,outcome
0,58.0,management,married,tertiary,no,2143,yes,no,unknown,5.0,may,261.0,1.0,224.577692,0,unknown,no
1,44.0,technician,single,secondary,no,29,yes,no,unknown,5.0,may,151.0,1.0,224.577692,0,unknown,no
2,33.0,entrepreneur,married,secondary,no,2,yes,yes,unknown,5.0,may,76.0,1.0,224.577692,0,unknown,no
3,47.0,blue-collar,married,unknown,no,1506,yes,no,unknown,5.0,may,92.0,1.0,224.577692,0,unknown,no
4,33.0,unknown,single,unknown,no,1,no,no,unknown,5.0,may,198.0,1.0,224.577692,0,unknown,no


In [8]:
# ord_df is has all categorical columns as 'ordinal' numbers.
ord_df = base_df.copy()
for col in cat_cols:
    ord_df[col] = ord_df[col].map(cat_map[col])

ord_df.head()

,age,job,marital,education,has_defaulted,balance,housing,loan,contact,day,month,duration,campaign,days_since_last_contact,prev_contact_count,prev_outcome,outcome
0,58.0,11,3,3,0,2143,1,0,0,5.0,5,261.0,1.0,224.577692,0,0,0
1,44.0,6,1,2,0,29,1,0,0,5.0,5,151.0,1.0,224.577692,0,0,0
2,33.0,7,3,2,0,2,1,1,0,5.0,5,76.0,1.0,224.577692,0,0,0
3,47.0,8,3,0,0,1506,1,0,0,5.0,5,92.0,1.0,224.577692,0,0,0
4,33.0,0,1,0,0,1,0,0,0,5.0,5,198.0,1.0,224.577692,0,0,0


In [9]:
# hot_df is the same as base_df but with all categorical columns replaced with
# one-hot encodings as floats
hot_df = pd.get_dummies(base_df, columns=cat_cols, dtype=float)
hot_df.head()

,age,balance,day,duration,campaign,days_since_last_contact,prev_contact_count,job_admin.,job_blue-collar,job_entrepreneur,...,prev_outcome_success,prev_outcome_unknown,has_defaulted_no,has_defaulted_yes,housing_no,housing_yes,loan_no,loan_yes,outcome_no,outcome_yes
0,58.0,2143,5.0,261.0,1.0,224.577692,0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,44.0,29,5.0,151.0,1.0,224.577692,0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,33.0,2,5.0,76.0,1.0,224.577692,0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,47.0,1506,5.0,92.0,1.0,224.577692,0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,33.0,1,5.0,198.0,1.0,224.577692,0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
